In [1]:
from UniCL import configs
import torch

In [4]:
unicl_model = torch.load('/data/michal5/unicl_model.pth')

In [5]:
print(type(unicl_model))

<class 'dict'>


In [6]:
print(unicl_model.keys())

dict_keys(['model'])


In [2]:
import UniCL

In [3]:
from UniCL.model import build_model

In [4]:
from UniCL.config import get_config

In [5]:
import argparse

In [6]:
parser = argparse.ArgumentParser('UniCL training and evaluation script', add_help=False)
parser.add_argument('--cfg', type=str, default='UniCL/configs/unicl_swin_base.yaml', metavar="FILE", help='path to config file', )
parser.add_argument(
    "--opts",
    help="Modify config options by adding 'KEY VALUE' pairs. ",
    default=None,
    nargs='+',
)

# easy config modification
parser.add_argument('--batch-size', type=int, help="batch size for single GPU")
parser.add_argument('--dataset', type=str, default='imagenet', help='dataset name')       
parser.add_argument('--data-path', type=str, help='path to dataset')
parser.add_argument('--zip', action='store_true', help='use zipped dataset instead of folder dataset')
parser.add_argument('--cache-mode', type=str, default='part', choices=['no', 'full', 'part'],
                    help='no: no cache, '
                         'full: cache all data, '
                         'part: sharding the dataset into nonoverlapping pieces and only cache one piece')
parser.add_argument('--resume', help='resume from checkpoint')
parser.add_argument('--accumulation-steps', type=int, help="gradient accumulation steps")
parser.add_argument('--use-checkpoint', action='store_true',
                    help="whether to use gradient checkpointing to save memory")
parser.add_argument('--amp-opt-level', type=str, default='O1', choices=['O0', 'O1', 'O2'],
                    help='mixed precision opt level, if O0, no amp is used')
parser.add_argument('--output', default='output', type=str, metavar='PATH',
                    help='root of output folder, the full path is <output>/<model_name>/<tag> (default: output)')
parser.add_argument('--tag', help='tag of experiment')
parser.add_argument('--eval', action='store_true', help='Perform evaluation only')
parser.add_argument('--throughput', action='store_true', help='Test throughput only')
parser.add_argument('--debug', action='store_true', help='Perform debug only')

# distributed training
parser.add_argument("--local_rank", type=int, required=False, help='local rank for DistributedDataParallel')




_StoreAction(option_strings=['--local_rank'], dest='local_rank', nargs=None, const=None, default=None, type=<class 'int'>, choices=None, help='local rank for DistributedDataParallel', metavar=None)

In [7]:
args, unparsed = parser.parse_known_args()

config = get_config(args)

=> merge config from UniCL/configs/unicl_swin_base.yaml


In [8]:
unicl_model = build_model(config)

Creating model: swin


/shared/rsaas/michal5/anaconda3/envs/must/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180594101/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
state_dict = torch.load('/data/michal5/unicl_model.pth')
unicl_model.load_state_dict(state_dict['model'])

<All keys matched successfully>

In [30]:
!  pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /srv/condor/execute/dir_76967/pip-req-build-htl3hqd_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /srv/condor/execute/dir_76967/pip-req-build-htl3hqd_
  Resolved https://github.com/openai/CLIP.git to commit d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Preparing metadata (setup.py) ... done


In [10]:
import clip

device = "cuda:0" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

In [11]:
from PIL import Image
import numpy as np
from torch.utils.data import Dataset
def pil_loader(path: str) -> Image.Image:
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
class FileListDataset(Dataset):
    def __init__(self, image_file, label_file, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        self.images = np.load(image_file)
        self.labels = np.load(label_file)
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        image = pil_loader(self.images[index])
        target = self.labels[index]

        if self.transform is not None:
            sample = self.transform(image)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

In [31]:
! ls /data/michal5/sun397

test_images.npy  train_images.npy  trainval_images.npy	val_images.npy
test_labels.npy  train_labels.npy  trainval_labels.npy	val_labels.npy


In [12]:
sun397_eval = FileListDataset('/data/michal5/sun397/test_images.npy', '/data/michal5/sun397/test_labels.npy', preprocess, target_transform=None)    


In [20]:
print(len(sun397_eval))

21758


In [13]:
unique_labels = set()
for l in sun397_eval.labels:
    unique_labels.add(l)
print(len(list(unique_labels)))

397


In [41]:
import json 
with open('/shared/rsaas/michal5/must/classes.json','r+') as f:
    cl = json.load(f)


In [45]:
device = 'cuda:0'
clip_text =torch.cat([clip.tokenize(f"a photo of a {c}") for c in cl['sun397']]).to(device)


In [46]:
from timm.utils import accuracy, AverageMeter

In [47]:
sun397_loader = torch.utils.data.DataLoader(sun397_eval,batch_size=512,shuffle=False)

NameError: name 'AverageMeter' is not defined

In [ ]:
from timm.utils import accuracy, AverageMeter
from tqdm import tqdm
unicl_model.eval()
clip_model.eval()
unicl_model = unicl_model.to('cuda:1')
correct = 0
total = 0
acc_meter = AverageMeter()
#unicl_embeddings = unicl_model.get_embeddings(unique_labels)

for (num,(images,labels)) in enumerate(tqdm(sun397_loader)):

    #print(images.size())
    #unicl 
    with torch.no_grad():
        text_features = clip_model.encode_text(clip_text)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        images = images.to('cuda:1')
        labels = labels.to('cuda:1')
        feat_img = unicl_model.encode_image(images)
#         #print(feat_img.size())
        text_features = text_features.to('cuda:1')
        unicl_model_output = unicl_model.logit_scale.exp() * feat_img.half() @ text_features.t()
# #         #clip 
        text_features = text_features.to('cuda:0')
        images = images.to('cuda:0')
        labels = labels.to('cuda:0')
        image_features = clip_model.encode_image(images)
        #text_features = clip_model.encode_text(clip_text)''
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logit_scale = clip_model.logit_scale.exp()

        similarity = logit_scale * image_features @ text_features.t()
        #values, ind = similarity.topk(1)

        
        
        
#         temp_c = 0
#         for i,l in zip(ind,labels):
#             if i.item() == l.item():
#                 temp_c += 1
#         print(temp_c,'temp')
            
#         print(labels.size(),'labels size')
#         c =  (ind == labels)
#         print(c,'new correct')
#         correct += c.sum().item()
#         print(correct, 'updated correct')
#         print(ind.size(),ind.size()[0],ind.size()[1])
        
# #         for i,l in zip(ind,labels):
# #             print(i.item(),l.item(),'hellp')
# #             if i == l:
# #                 print('yes')
# #                 correct+=1
# # #         temp_acc = (ind==labels).sum().item()
# # #         correct += temp_acc 
# #         print(ind.size()[1],'size')
#         total+= ind.size()[0]
#         print(total)
        #print(temp_acc,'temp acc')
        unicl_model_output = unicl_model_output.to('cuda:0')
        combined_output = (unicl_model_output+similarity)
        #combined_output = similarity
        #print(similarity.softmax(dim=-1).topk(0),labels)
        #combined_output = com
        acc = accuracy(combined_output,labels,topk=(1,))
        print(acc,'acc')
        acc_meter.update(acc[0].item(), labels.size(0))
        print(acc_meter.avg,'avg')

    


  2%|████▎                                                                                                                                                                                     | 1/43 [00:16<11:32, 16.49s/it]

[tensor(3.9062, device='cuda:0')] acc
3.90625 avg


  5%|████████▋                                                                                                                                                                                 | 2/43 [00:32<11:10, 16.35s/it]

[tensor(0., device='cuda:0')] acc
1.953125 avg


  7%|████████████▉                                                                                                                                                                             | 3/43 [00:53<12:13, 18.35s/it]

[tensor(0., device='cuda:0')] acc
1.3020833333333333 avg


In [51]:
print(acc_meter.avg)

1.245959051724138
